In [1]:
# Step 1: Clone the LLaMA-Factory repository (if not done already)
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (315/315), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 315 (delta 80), reused 160 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (315/315), 8.97 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py


In [2]:
# Step 2: Install the necessary packages
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]
!pip install git+https://github.com/huggingface/transformers
!pip install liger-kernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.

In [3]:
# Step 3: Set up Huggingface authentication (if not done already)
from google.colab import userdata
HF_TOKEN2 = userdata.get('HF_TOKEN')
!huggingface-cli login --token $HF_TOKEN2 --add-to-git-credential

Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [23]:
# Step 4: Create DPO training configuration
import json

args_dpo = dict(
  stage="dpo",
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit",
  dataset="alpaca_en_demo",
  template="llama3",
  finetuning_type="lora",
  output_dir="llama3_dpo",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_samples=500,
  max_grad_norm=1.0,
  fp16=True
)

json.dump(args_dpo, open("train_llama3_dpo.json", "w", encoding="utf-8"), indent=2)

In [6]:
!pip uninstall -y transformers

Found existing installation: transformers 4.46.0.dev0
Uninstalling transformers-4.46.0.dev0:
  Successfully uninstalled transformers-4.46.0.dev0


In [7]:
!pip install "transformers>=4.41.2,<4.45.0"

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0


In [15]:
# Check the available datasets in dataset_info.json
!cat /content/LLaMA-Factory/src/llamafactory/data/dataset_info.json


cat: /content/LLaMA-Factory/src/llamafactory/data/dataset_info.json: No such file or directory


In [26]:
# Step 1: Modify the DPO JSON file to remove the training stage for flexibility and set a compatible dataset
import json

# Load the DPO training configuration file
with open("/content/LLaMA-Factory/train_llama3_dpo.json", "r") as f:
    args_dpo = json.load(f)

# Set the dataset to 'alpaca_en_demo'
args_dpo['dataset'] = "alpaca_en_demo"

# Remove the stage key to default to DPO stage
if 'stage' in args_dpo:
    del args_dpo['stage']

# Save the updated JSON configuration back to the file
with open("/content/LLaMA-Factory/train_llama3_dpo.json", "w") as f:
    json.dump(args_dpo, f, indent=2)

# Step 2: Re-run the DPO training process
%cd /content/LLaMA-Factory/
!llamafactory-cli train train_llama3_dpo.json


/content/LLaMA-Factory
2024-10-03 08:28:30.760200: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 08:28:30.793396: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 08:28:30.803635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 08:28:30.828217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 08:28:32.13